# EDA India

## Overview
This notebook contains **Exploratory Data Analysis** and **Visualization** for the cases and vaccinations in India.

**Sections:**
1. [Data Ingestion](#Data_Ingestion)
2. [Summary Statistics](#Summary_Statistics)
3. [Data Cleaning](#3.-Data-Cleaning)
4. [Visualization](#Visualization)
5. [Correlation](#Correlation)
6. [Preprocessing](#Preprocessing)
7. [Conclusion](#Conclusion)

***
## Setup 

In [ ]:
!pip install seaborn

In [ ]:
import os
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.interpolate import CubicSpline

***
<a id='Data_Ingestion'></a>
## 1. Data Ingestion

### 1.1 Getting daily state-wise cases for India

In [ ]:
cases = pd.read_csv("india_cases_12-10-2021.csv", index_col=0)
cases

### 1.2  Getting daily state-wise vaccinations for India

In [ ]:
vacc = pd.read_csv("india_vaccines_12-10-2021.csv", index_col=0)
vacc

***
<a id='Summary_Statistics'></a>
## 2. Summary statistics

In [ ]:
# Summary statistics for cases in India
cases.describe()

In [ ]:
# Summary statistics for vaccines in India
vacc.describe()

<a id='missing_outliers'></a>
### 2.2 How many missing data and outliers? 

In [ ]:
# Provides the number of missing values for cases in India
cases.isnull().sum()

In [ ]:
cases.isnull().sum().sum()

In [ ]:
# Provides the number of missing values for vaccines in India
vacc.isnull().sum()

In [ ]:
vacc.isnull().sum().sum()

In [ ]:
z_confirm = np.abs(stats.zscore(cases['Confirmed']))
print(z_confirm)

In [ ]:
z_adminstered = np.abs(stats.zscore(vacc['Total Doses Administered']))
print(z_adminstered)

In [ ]:
threshold = 3
print(np.where(z_confirm>3))

In [ ]:
np.where(z_adminstered>3)

### 2.3 Any Inconsistent, Incomplete, duplicate or incorrect data

In [ ]:
cases.duplicated().sum()

In [ ]:
vacc.duplicated().sum()

In [ ]:
incomplete_cases = cases.isnull().any(axis=1)
incomplete_cases

In [ ]:
incomplete_vacc = vacc.isnull().any(axis=1)
incomplete_vacc

***
## 3. Data Cleaning

### 3.1 cases 

In [ ]:
cases.info()

In [ ]:
# Provides the fraction of nulls in a particular column
cases.isnull().sum() / len(cases)

Above we can see that `Province/State`, `FIPS`, `Admin2` have a Null Ratio of `1.00` 
So we can go ahead and drop these columns from our dataframe

In [ ]:
#Dropping the columns with 100% null values
cases.drop(['Province/State', 'FIPS', 'Admin2'], inplace = True, axis = 1)

`Last Update`, `Latitude` and `Longitude` have very few non-null values. This is because they are repeated columns corresponding `Last_Update`, `Lat` and `Long_` which have a low Null Ratio. Due to this we have decided to merge the values from the first set *(High Null Ratio)* to its corresponding second set column *(Low Null Ratio)*

In [ ]:
# Number of nulls in second set columns before combining
print(cases['Last_Update'].isnull().sum())
print(cases['Lat'].isnull().sum())
print(cases['Long_'].isnull().sum())

In [ ]:
# For Last Update and Last_Update
cases['Last_Update'] = cases['Last_Update'].combine_first(cases['Last Update'])

# For Latitude and Lat
cases['Lat'] = cases['Lat'].combine_first(cases['Latitude'])

# For Longitude and Long
cases['Long_'] = cases['Long_'].combine_first(cases['Longitude'])

In [ ]:
# Number of nulls in second set columns after combining
print(cases['Last_Update'].isnull().sum())
print(cases['Lat'].isnull().sum())
print(cases['Long_'].isnull().sum())

Now that `Last Update`, `Latitude` and `Longitude` have been combined into their corresponding `Last_Update`, `Lat` and `Long_` columns, the first set is now redundant. So we can drop them

In [ ]:
cases.drop(['Last Update', 'Latitude', 'Longitude'], inplace=True, axis = 1)

In [ ]:
# Info after cleaning cases
cases.info()

### 3.2 vacc

In [ ]:
vacc.info()

In [ ]:
# Provides the fraction of nulls in a particular column
vacc.isnull().sum() / len(vacc)

There is no column with a significant majority of Nulls. **All columns can be left as is**

***
<a id='Visualization'></a>
## 4. Visualization

### 4.1 Histograms

In [ ]:
histogram_filter_cases = cases[['Confirmed', 'Recovered']]
sns.histplot(data=histogram_filter_cases, bins=30, kde=True)

In [ ]:
histogram_filter_cases = cases[['Deaths']]
sns.histplot(data=histogram_filter_cases, bins=30, kde=True)

### 4.2 Bar Charts

In [ ]:
barchart_filter_cases = cases[['Confirmed', 'Recovered','Deaths']]
sns.set_theme(style='whitegrid')
sns.barplot(data=barchart_filter_cases)

In [ ]:
barchart_filter_vacc = vacc[['First Dose Administered', 'Second Dose Administered']]
sns.set_theme(style='whitegrid')
sns.barplot(data=barchart_filter_vacc)

In [ ]:
barchart_filter_vacc = vacc[['18-44 Years (Doses Administered)', '45-60 Years (Doses Administered)', '60+ Years (Doses Administered)']]
sns.set_theme(style='whitegrid')
b = sns.barplot(data=barchart_filter_vacc)
b.set(xticklabels=['18-44 Years','45-60 Years','60+ Years'])

### 4.3 Line Plots 

#### Getting Time Series Data
Before obtaining Line Plots we first extract the time series data. This is done by
- Grouping by Date
- Aggregating Confirmed, Deaths and Recovered by Sum

In [ ]:
# Cumulative Time Series (Add on to previous date's cases)
cum_timeseries = cases.groupby(['Date']).agg(Confirmed = ('Confirmed', 'sum'), Deaths = ('Deaths', 'sum'), Recovered = ('Recovered', 'sum'))

# Delta Time series (Cases on a particular day)
delta_timeseries = cum_timeseries.diff()

### Plots

**1. Confirmed Cases**

1.1 Cumulative confirmed cases

In [ ]:
cum_timeseries.Confirmed.plot(figsize=(8, 8))

1.2 Daily confirmed cases

In [ ]:
delta_timeseries.Confirmed.plot(figsize=(8, 8))

**2. Deaths**

2.1 Cumulative Deaths

In [ ]:
cum_timeseries.Deaths.plot(figsize=(8, 8))

2.1 Cumulative Deaths

In [ ]:
delta_timeseries.Deaths.plot(figsize=(8, 8))

**3. Recovered**    
***NOTE:*** Recovered data is only available upto `2021-08-04` and is missing after that date. So the plot has been done only considering dates before `2021-08-05`, after cleaning the data

In [ ]:
# Trimming off NaNs in Recovered Column after 2021-08-04 and then getting aggregate count by date
trim_df = cases['Date'] < '2021-08-05'
trim_cases = cases.loc[trim_df] 
rec_cum_timeseries = trim_cases.groupby(['Date']).agg(Recovered = ('Recovered', 'sum'))

In [ ]:
rec_cum_timeseries.Recovered.plot(figsize=(8, 8))

In [ ]:
rec_delta_timeseries = rec_cum_timeseries.diff()
rec_delta_timeseries.Recovered.plot(figsize=(8, 8))

### 4.4 PCA

#### 4.1 Convert categorical columns to numerical

***
<a id='Correlation'></a>

## 5. Correlation

- Find the correlation matrix for `cases` and `vacc` to see if any attributes are strongly correlated (we take a threshold of 80%)
- We see if the correlation is meaningful, or indicative of excessive attributes

### 5.1 Correlation for `cases`

In [ ]:
# Correlation matrix
corr_cases = cases.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr_cases, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(corr_cases, mask=mask, center=0, square=True, annot=True)

### Observations - `cases`

1. We observe that `Deaths` and `Confirmed` are stronly positively correlated with a correlation coefficient of `0.91`
2. We also observe that `Deaths` and `Recovered` are stronly positively correlated with a correlation coefficient of `0.92`
3. Finally, we observe that `Recovered` and `Confirmed` are stronly positively correlated with a correlation coefficient of `1`
4. There is no strong negative correlation between any of the attributes

These observations are consistent with the expectation that as daily case numbers rise, recovered numbers and death counts rise

### 5.2 Correlation for `vacc`

In [ ]:
# Correlation matrix
corr_vacc = vacc.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr_vacc, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))

sns.heatmap(corr_vacc, mask=mask, center=0, square=True, annot=True)

### Observations - `vacc`

Almost all the attributes are strongly positively correlated. This makes intuitive sense as many of the attributes are subsets of the total doses administered in a single day.

***
<a id='Preprocessing'></a>
## 6. Preprocessing

1. Continuum cubic spline curve interpolation

### 6.1  Contituum cubic spline curve interpolation on 7-day average

- The 7-day average for total daily number of cases is calculated
- The values are then interpolated using a cubic spline curve
- This is done to create a continuous function out of discrete points
- The derivatives of the CS curve are plotted for smoothed out `Confirmed`, `Deaths` and `Recovered`

#### 6.1.1 Seven-day average of daily confirmed cases

In [ ]:
# Find 7 day average of confirmed cases from delta_timeseries

confirmed_7_day = delta_timeseries.Confirmed.rolling(7, min_periods=1).mean()
confirmed_7_day = confirmed_7_day.reset_index()

confirmed_7_day

#### Lineplot of smoothed (7-day-average) confirmed cases

In [ ]:
confirmed_7_day.plot(x='Date', figsize=(8, 8))

#### Cubic Spline Curve Interpolation

In [ ]:
confirmed_7_day = confirmed_7_day.loc[confirmed_7_day['Confirmed'].notnull()]
confirmed_7_day['Date_delta'] = (pd.to_datetime(confirmed_7_day['Date']) - pd.to_datetime(confirmed_7_day['Date']).min())  / np.timedelta64(1,'D')

cs_confirmed = CubicSpline(confirmed_7_day.Date_delta, confirmed_7_day.Confirmed)

#### Plot of DC (Daily Confirmed)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(confirmed_7_day['Date_delta'], cs_confirmed(confirmed_7_day['Date_delta']), label='Daily cases (DC)')

#### Plot of DC' (First derivative of Daily Confirmed)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(confirmed_7_day['Date_delta'], cs_confirmed(confirmed_7_day['Date_delta'], 1), label='First derivative of DC')

#### 6.1.2 Seven-day average of daily deaths

In [ ]:
# Find 7 day average of deaths from delta_timeseries

deaths_7_day = delta_timeseries.Deaths.rolling(7, min_periods=1).mean()
deaths_7_day = deaths_7_day.reset_index()

deaths_7_day

#### Lineplot of smoothed (7-day-average) deaths

In [ ]:
deaths_7_day.plot(x='Date', figsize=(8, 8))

#### Cubic Spline Curve Interpolation

In [ ]:
deaths_7_day = deaths_7_day.loc[deaths_7_day['Deaths'].notnull()]
deaths_7_day['Date_delta'] = (pd.to_datetime(deaths_7_day['Date']) - pd.to_datetime(deaths_7_day['Date']).min())  / np.timedelta64(1,'D')

cs_deaths = CubicSpline(deaths_7_day.Date_delta, deaths_7_day.Deaths)

#### Plot of DD (Daily Deaths)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(deaths_7_day['Date_delta'], cs_deaths(deaths_7_day['Date_delta']), label='Daily Deaths (DD)')

#### Plot of DD' (First derivative of DD)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(deaths_7_day['Date_delta'], cs_deaths(deaths_7_day['Date_delta'], 1), label='First derivative of DD')

#### 6.1.3 Seven-day average of daily recoveries

In [ ]:
# Find 7 day average of recovered cases from delta_timeseries

rec_7_day = rec_delta_timeseries.Recovered.rolling(7, min_periods=1).mean()
rec_7_day = rec_7_day.reset_index()

rec_7_day

#### Lineplot of smoothed (7-day-average) recoveries

In [ ]:
rec_7_day.plot(x='Date', figsize=(8, 8))

#### Cubic Spline Curve Interpolation

In [ ]:
rec_7_day = rec_7_day.loc[rec_7_day['Recovered'].notnull()]
rec_7_day['Date_delta'] = (pd.to_datetime(rec_7_day['Date']) - pd.to_datetime(rec_7_day['Date']).min())  / np.timedelta64(1,'D')

cs_rec = CubicSpline(rec_7_day.Date_delta, rec_7_day.Recovered)

#### Plot of DR (Daily Recoveries)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(rec_7_day['Date_delta'], cs_rec(rec_7_day['Date_delta']), label='Daily Recoveries (DR)')

#### Plot of DR' (First derivative of DR)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(rec_7_day['Date_delta'], cs_rec(rec_7_day['Date_delta'], 1), label='First derivative of DR')

***
<a id='Conclusion'></a>
## 7. Conclusion 

- How many rows and attributes?
    - Size of cases in india is: `(18214, 22)`.
    - Size of vaccinations in india is: `(9990, 24)`.
- How many missing data and outliers?
    - `153446` missing elements for cases.
    - `73305` missing elements for vaccines.
- Any inconsistent, incomplete, duplicate or incorrect data?
    - All values in cases and vaccines are unique.
    - Cases in india contains `18214` incomplete rows.
    - Vaccinations in india contains `9990` incomplete rows.
- Are the variables correlated to each other?
    - The `Recovered`, `Confirmed` and `Deaths` columns in `cases` are strongly positively correlated.
    - Almost all attributes in `vacc` are strongly positively correlated.

- Are any of the preprocessing techniques needed: rolling average, continuum cubic spline curve, dimensionality reduction, range transformation, standardization, etc.?
    - Using the 7-day rolling average of the daily `Confirmed`, `Deaths` and `Recovered` shows a smoother plot with less noise.
    - Calculating the continuum cubic spline curve allows us to differentiate the function and find the instantaneous slope.

- Does PCA help visualize the data? Do we get any insights from histograms/bar charts/line plots, etc.?